In [55]:
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
from bibtexparser import dumps
import requests
import base64
import matplotlib.pyplot as plt
from IPython.core.pylabtools import print_figure


class Works:
    def __init__(self, oaid):
        self.oaid = oaid
        self.req = requests.get(f'https://api.openalex.org/works/{oaid}')
        self.data = self.req.json()
        
    def __str__(self):
        return 'str'
        
    def __repr__(self):
        return 'repr'

    @property
    def bibtex(self):
        _authors = [au['author']['display_name'] for au in self.data['authorships']]
        if len(_authors) == 0:
            authors = 'Editorial'
        elif len(_authors) == 1:
            authors = _authors[0]
        else:
            authors = ', '.join(_authors[0:-1]) + ' and' + _authors[-1] 
        
        pages = '-'.join([self.data['biblio'].get('first_page', '') or '',
                          self.data['biblio'].get('last_page', '') or ''])
        
        db = BibDatabase()
        db.entries = [{
            'ID': self.data['id'],
            'title': self.data['title'],
            'author': authors,
            'journal': self.data['host_venue']['display_name'],
            'volume': self.data['biblio']['volume'],
            'number': self.data['biblio']['issue'],
            'pages': pages,
            'year': str(self.data['publication_year']),
            'url': self.data['doi'],
            'ENTRYTYPE': self.data['type']
        }]
        writer = BibTexWriter()
        bibtex = dumps(db, writer)
        return print(bibtex)
    
    @property
    def ris(self):
        fields = []
        if self.data['type'] == 'journal-article':
            fields += ['TY  - JOUR']
        else:
            raise Exception("Unsupported type {self.data['type']}")
        
        for author in self.data['authorships']:
            fields += [f'AU  - {author["author"]["display_name"]}']
            
        fields += [f'PY  - {self.data["publication_year"]}']
        fields += [f'TI  - {self.data["title"]}']
        fields += [f'JO  - {self.data["host_venue"]["display_name"]}']
        fields += [f'VL  - {self.data["biblio"]["volume"]}']
        
        if self.data['biblio']['issue']:
            fields += [f'IS  - {self.data["biblio"]["issue"]}']
        
        
        fields += [f'SP  - {self.data["biblio"]["first_page"]}']
        fields += [f'EP  - {self.data["biblio"]["last_page"]}']
        fields += [f'DO  - {self.data["doi"]}']
        fields += ['ER  -']
                
        ris = '\n'.join(fields)
        return print(ris)

In [56]:
w = Works('https://doi.org/10.1021/acscatal.5b00538')

In [57]:
w.bibtex

@journal-article{https://openalex.org/W2288114809,
 author = {John R. Kitchin},
 journal = {ACS Catalysis},
 number = {6},
 pages = {3894-3899},
 title = {Examples of Effective Data Sharing in Scientific Publishing},
 url = {https://doi.org/10.1021/acscatal.5b00538},
 volume = {5},
 year = {2015}
}



In [58]:
w.ris

TY  - JOUR
AU  - John R. Kitchin
PY  - 2015
TI  - Examples of Effective Data Sharing in Scientific Publishing
JO  - ACS Catalysis
VL  - 5
IS  - 6
SP  - 3894
EP  - 3899
DO  - https://doi.org/10.1021/acscatal.5b00538
ER  -
